In [1]:
import torch
from torchessian import hessian_matmul, lanczos
from torchvision.models import resnet18
from itertools import product

In [2]:
n = 2
model = torch.nn.Linear(n, 1)
# torch.manual_seed(2019)
v = torch.rand(sum(p.data.numel() for p in model.parameters()))
x = torch.rand(10, n)
y = x.sum(1)
def loss_function(y_hat, y):
    return ((y_hat.view(-1) - y.view(-1)) * (y_hat.view(-1) - y.view(-1))).sum() / y.numel()
batch = x, y
m = 3
# Exact Hessian

h11 = 2 * (x[:, 0] * x[:, 0]).sum() / x.size(0)
h21 = h12 = 2 * (x[:, 0] * x[:, 1]).sum() / x.size(0)
h31 = h13 = 2 * x[:, 0].sum() / x.size(0)

h22 = 2 * (x[:, 1] * x[:, 1]).sum() / x.size(0)
h32 = h23 = 2 * x[:, 1].sum() / x.size(0)

h33 = 2

H = torch.zeros(3, 3)

for i, j in product(range(3), range(3)):
    exec("H[%d, %d] = h%d%d" % (i, j, i+1, j+1))
    
print(H)

tensor([[0.6816, 0.6491, 1.0284],
        [0.6491, 0.9645, 1.2512],
        [1.0284, 1.2512, 2.0000]])


In [3]:
T, V = lanczos(model, loss_function, batch, m)

In [4]:
v = torch.eig(T)[0]
v.sort()
v

tensor([[3.3829, 0.0000],
        [0.1022, 0.0000],
        [0.1605, 0.0000]])

In [5]:
v = torch.eig(H)[0]
v.sort()
v

tensor([[3.3834, 0.0000],
        [0.1606, 0.0000],
        [0.1021, 0.0000]])

In [6]:
for i in range(m - 1):
    print("{:.3f}".format(V[i, :].dot(V[m - 1, :])))

-0.000
0.000
